In [2]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 500)

In [3]:
POSSIBLE_VALUES = [2,3,4,5,6,7,8,9,10,11]

DECK_COLUMNS = [str(i) for i in POSSIBLE_VALUES]
HAND_COLUMNS = ["score", "cards", "aces"]

In [4]:
GAME_COLUMNS = ["chance"] + ["player_"+colname for colname in HAND_COLUMNS] + ["dealer_"+colname for colname in HAND_COLUMNS] + DECK_COLUMNS

# SETUP

In [5]:
def get_deck(number_decks=6):
    """get a deck with the specified number of decks (default: 6)"""
    
    deck = pd.Series([number_decks*4]*len(DECK_COLUMNS) ,index=DECK_COLUMNS)
    deck.loc["10"] *= 4
    return deck

In [6]:
def get_game(deck=None, player_cards=[], dealer_cards=[], update_deck=True):
    """get a game with the specified deck, player cards and dealer cards"""

    if deck is None:
        deck = get_deck()
    
    game = pd.DataFrame(
        [1.0] + [0, 0, 0]*2 + deck.values.flatten().tolist(), 
        index=GAME_COLUMNS
    ).T.astype(int).astype({"chance":float})

    for player_card_value in player_cards:
        game = update_player_with_card(game, player_card_value, probabilistic=False, update_deck=update_deck)
    
    for dealer_card_value in dealer_cards:
        game = update_dealer_with_card(game, dealer_card_value, probabilistic=False, update_deck=update_deck)
    
    return game

In [7]:
def update_player_with_card(game, value, probabilistic=True, update_deck=True):
    """update the game with the player drawing a card with the specified value"""

    assert np.all(game["player_score"] >= 0) and np.all(game["player_score"] != 22), f"there are finished hands, but player tried to add card {value}. hands:\n{game}"

    next_game = game.copy()

    if probabilistic:
        # updating chance: multiplying by the probability of drawing that value
        next_game["chance"] *= (next_game[f"{value}"]/next_game[DECK_COLUMNS].sum(axis=1))
    if update_deck:
        # update deck info
        next_game[f"{value}"] -= 1

    # updating score
    next_game["player_score"] += value
    # updating cards
    next_game["player_cards"] += 1
    # updating aces
    next_game["player_aces"] += (value == 11)

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["player_score"] > 21) & (next_game["player_aces"] > 0)
    next_game.loc[acenecessary,"player_score"] -= 10
    next_game.loc[acenecessary,"player_aces"] -= 1

    # set bust to -2
    next_game.loc[next_game["player_score"] > 21, "player_score"] = -2

    # set blackjack to 22
    next_game.loc[(next_game["player_score"] == 21) & (next_game["player_cards"] == 2), "player_score"] = 22

    return next_game

In [8]:
def update_dealer_with_card(game, value, probabilistic=True, update_deck=True):
    """update the game with the dealer drawing a card with the specified value"""

    assert np.all(game["dealer_score"] >= 0) and np.all(game["dealer_score"] < 17), f"there are finished hands, but dealer tried to add card {value}. hands:\n{game}"

    next_game = game.copy()

    if probabilistic:
        # updating chance: multiplying by the probability of drawing that value
        next_game["chance"] *= (next_game[f"{value}"]/next_game[DECK_COLUMNS].sum(axis=1))
    if update_deck:
        # update deck info
        next_game[f"{value}"] -= 1

    # updating score
    next_game["dealer_score"] += value
    # updating cards
    next_game["dealer_cards"] += 1
    # updating aces
    next_game["dealer_aces"] += (value == 11)

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["dealer_score"] > 21) & (next_game["dealer_aces"] > 0)
    next_game.loc[acenecessary,"dealer_score"] -= 10
    next_game.loc[acenecessary,"dealer_aces"] -= 1

    # set bust to -1
    next_game.loc[next_game["dealer_score"] > 21, "dealer_score"] = -1 

    # set blackjack to 22
    next_game.loc[(next_game["dealer_score"] == 21) & (next_game["dealer_cards"] == 2), "dealer_score"] = 22

    return next_game

In [9]:
def order_game_columns(game):
    """order the columns of the game dataframe"""

    return game[GAME_COLUMNS + [colname for colname in game.columns if colname not in GAME_COLUMNS]]

# PLAYER

In [10]:
def player_hits_once(game, keep_drawn_column=False):
    """get updated game when player hits once, throws error for finished hands"""

    assert np.all(game["dealer_cards"] != 0), f"dealer must have one card showing. hands: {game}"    
    assert np.all(game["player_score"] >= 0) and np.all(game["player_score"] != 22), f"there are finished hands, but player tried to hit. hands:\n{game}"
    assert game[DECK_COLUMNS].sum(axis=1).max() == game[DECK_COLUMNS].sum(axis=1).min(), f"there are decks with different number of cards. hands:\n{game}"
    
    # add possible cards to draw
    next_game = pd.merge(game, pd.DataFrame(np.arange(2,12), columns=["drawn"]), how="cross")

    # helper diag matrix
    diag = np.tile(np.diag(np.ones(10)).astype(bool), (len(game),1))

    # update drawing chance
    next_game["chance"] *= next_game[DECK_COLUMNS].values[diag] / next_game[DECK_COLUMNS].sum(axis=1).values

    # update deck info
    next_game.loc[:,DECK_COLUMNS] -= diag.astype(int)

    # updating score
    next_game["player_score"] += next_game["drawn"]
    # updating cards
    next_game["player_cards"] += 1
    # updating aces
    next_game.loc[next_game["drawn"]== 11, "player_aces"] += 1

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["player_score"] > 21) & (next_game["player_aces"] > 0)
    next_game.loc[acenecessary,"player_score"] -= 10
    next_game.loc[acenecessary,"player_aces"] -= 1

    # set bust to -2
    next_game.loc[next_game["player_score"] > 21, "player_score"] = -2

    # set blackjack to 22
    next_game.loc[(next_game["player_score"] == 21) & (next_game["player_cards"] == 2), "player_score"] = 22

    # drop drawn column
    if not keep_drawn_column:
        next_game = next_game.drop(columns=["drawn"])

    # group and reorder
    next_game = next_game.groupby(next_game.columns.tolist()[1:]).agg({'chance': 'sum'}).reset_index()
    next_game = order_game_columns(next_game)

    return next_game.sort_values("player_score")


In [11]:
def player_hits_once_where_possible(game, verbose=False):
    """get updated game when player hits once, finished hands are kept as-is"""

    assert np.all(game["dealer_cards"] == 1), f"dealer must have one card showing. hands:\n{game}"  

    is_playable = (game["player_score"] >= 0) & (game["player_score"] != 22)
    if is_playable.sum() == 0:
        if verbose : print("All possible games are finished")
        return game
    elif is_playable.sum() > 0:
        if verbose : print(f"Only {is_playable.sum()} games are playable")
    
    final = game[~is_playable].copy()
    playable = game[is_playable].copy()

    next_possible = player_hits_once(playable)
    final = pd.concat((final, next_possible), axis=0, ignore_index=True)

    # group and reorder
    final = final.groupby(final.columns.tolist()[1:]).agg({'chance': 'sum'}).reset_index()
    final = order_game_columns(final)

    return final.sort_values("player_score")

# DEALER

In [12]:
def dealer_draws_once(game):
    """get updated game when dealer draws one card, throws error for finished hands"""

    assert np.all(game["dealer_score"] >= 0) and np.all(game["dealer_score"] < 17), f"there are finished hands, but dealer tried to draw. hands:\n{game}"
    assert "drawn" not in game.columns, f"drawn column already exists. hands:\n{game}"
    
    # add possible cards to draw
    next_game = pd.merge(game, pd.DataFrame(np.arange(2,12), columns=["drawn"]), how="cross")

    # helper diag matrix
    diag = np.tile(np.diag(np.ones(10)).astype(bool), (len(game),1))

    # update drawing chance
    next_game["chance"] *= next_game[DECK_COLUMNS].values[diag] / next_game[DECK_COLUMNS].sum(axis=1).values

    # update deck info
    next_game.loc[:,DECK_COLUMNS] -= diag.astype(int)

    # updating score
    next_game["dealer_score"] += next_game["drawn"]
    # updating cards
    next_game["dealer_cards"] += 1
    # updating aces
    next_game.loc[next_game["drawn"]== 11, "dealer_aces"] += 1

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["dealer_score"] > 21) & (next_game["dealer_aces"] > 0)
    next_game.loc[acenecessary,"dealer_score"] -= 10
    next_game.loc[acenecessary,"dealer_aces"] -= 1

    # set bust to -2
    next_game.loc[next_game["dealer_score"] > 21, "dealer_score"] = -1

    # set blackjack to 22
    next_game.loc[(next_game["dealer_score"] == 21) & (next_game["dealer_score"] == 2), "dealer_score"] = 22

    # removing drawn column
    next_game = next_game.drop("drawn", axis="columns")
    
    next_game = next_game.groupby(game.columns.tolist()[1:]).agg({'chance': 'sum'}).reset_index()

    next_game = order_game_columns(next_game)

    return next_game.sort_values("dealer_score")

In [13]:
def dealer_plays(game):
    """get updated game when dealer plays until 17 or bust"""

    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"

    is_playable = (game["dealer_score"] >= 0) & (game["dealer_score"] < 17)
    final = game[~is_playable]
    playable = game[is_playable]

    if len(playable) == 0:
        if verbose : print("Dealer has finished all hands")
        return game

    while len(playable) > 0:
        next_possible = dealer_draws_once(playable)

        is_playable = (next_possible["dealer_score"] < 17) & (next_possible["dealer_score"] >= 0)
        # finished hands are taken out of the game
        final = pd.concat((final, next_possible[~is_playable]), axis=0, ignore_index=True)
        # playable hands
        playable = next_possible[is_playable]
    
    return final

# RETURNS

In [14]:
def get_return_finished(game):
    """get the probabilities of each possible return of the current state for the specified game
    no further moves are made"""

    assert np.all((game["dealer_score"] < 0) | (game["dealer_score"] >= 17)), "dealer has not finished yet"
    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"

    # blackjack win
    blackjack_win = (game["player_score"] == 22) & (game["dealer_score"] != 22)
    blackjack_win_chance = game.loc[blackjack_win, "chance"].sum()

    # win
    win = (game["player_score"] > game["dealer_score"]) & (game["player_score"] != 22)
    win_chance = game.loc[win, "chance"].sum()

    # lose
    lose = (game["player_score"] < game["dealer_score"])
    lose_chance = game.loc[lose, "chance"].sum()

    # push
    push = (game["player_score"] == game["dealer_score"])
    push_chance = game.loc[push, "chance"].sum()

    expected_return = blackjack_win_chance*1.5 + win_chance*1 + push_chance*0 + lose_chance*(-1)

    return pd.Series({
        "blackjack_win_chance": blackjack_win_chance,
        "win_chance": win_chance,
        "push_chance": push_chance,
        "lose_chance": lose_chance,
        "expected_return": expected_return
    })

In [15]:
def get_return_standing(game, verbose=False):
    """get the expected return of standing (or doing nothing in case the player is already finished)"""

    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"
    assert np.all(game["dealer_cards"] == 1), "Dealer must have exactly one card showing."

    game = dealer_plays(game.copy())

    if verbose : print(game)

    return get_return_finished(game)

In [16]:
def get_return_hitting_once(game, verbose=False):
    """get the expected return of hitting exactly once, throws error for finished hands"""
    
    assert np.all(game["player_score"] < 22), "player has a natural"
    assert np.all(game["player_score"] >= 0), "player is already busted"
    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"
    assert np.all(game["dealer_cards"] == 1), "Dealer must have exactly one card showing."

    game = player_hits_once(game.copy())

    return get_return_standing(game, verbose=verbose)

In [17]:
def get_return_hitting_once_where_possible(game, verbose=False):
    """get the expected return of hitting exactly once, finished hands are kept as-is"""

    assert np.all(game["dealer_cards"] == 1), f"dealer must have one card showing. hands:\n{game}"  
    
    game = player_hits_once_where_possible(game.copy())

    return get_return_standing(game, verbose=verbose)

In [18]:
def get_return_doubling_down(game, verbose=False):
    """get the expected return of doubling down, throws error for finished hands"""

    assert np.all(game["player_score"] < 22), "player has a natural"
    assert np.all(game["player_score"] >= 0), "player is already busted"
    assert np.all(game["player_cards"] == 2), "player does not have exactly 2 cards"
    assert np.all(game["dealer_cards"] == 1), "Dealer must have exactly one card showing."

    game = player_hits_once(game.copy())

    return_chances = get_return_standing(game, verbose=verbose)
    return_chances["expected_return"] *= 2
    
    return return_chances

In [19]:
def get_return_doubling_down_where_possible(game, verbose=False):
    """get the expected return of doubling down, finished hands are kept as-is"""

    assert np.all(game["dealer_cards"] == 1), f"dealer must have one card showing. hands:\n{game}"  
    
    is_playable = (game["player_score"] >= 0) & (game["player_score"] != 22)
    if is_playable.sum() == 0:
        print("All possible games are finished")
        return game
    elif is_playable.sum() > 0:
        print(f"Only {is_playable.sum()} games are playable")
    
    final = game[~is_playable].copy()
    playable = game[is_playable].copy()

    next_possible = player_hits_once(playable)
    final = pd.concat((final, next_possible), axis=0, ignore_index=True)

    return_chances = get_return_standing(final, verbose=verbose)
    return_chances["expected_return"] *= 2
    
    return return_chances

In [68]:
def get_return_column(game):
    """get column denoting the return for each game, assuming its finished"""

    assert np.all((game["dealer_score"] < 0) | (game["dealer_score"] >= 17)), "dealer has not finished yet"
    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"

    return_column = pd.Series(np.zeros(len(game)))

    return_column[(game["player_score"] == 22) & (game["dealer_score"] != 22)] = 1.5
    return_column[(game["player_score"] > game["dealer_score"]) & (game["player_score"] != 22)] = 1
    return_column[game["player_score"] == game["dealer_score"]] = 0
    return_column[game["player_score"] < game["dealer_score"]] = -1

    return return_column

# TESTING

In [69]:
def get_return_best_move(game, verbose=False):
    """get the expected return of hitting, taking into account that you could hit multiple times"""

    assert np.all(game["player_score"] < 22), "player has a natural"
    assert np.all(game["player_score"] >= 0), "player is already busted"
    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"
    assert np.all(game["dealer_cards"] == 1), "Dealer must have exactly one card showing."


    # expected return of standing
    print("simulating stand")
    standing_return = get_return_standing(game, verbose=verbose)
    print("standing return\n", standing_return)
    # this is the score to beat. lets see if hitting makes it better

    is_playable = (playable["player_score"] >= 0) & (playable["player_score"] != 22)
    playable = game[is_playable].copy()
    hits = 0


    while len(playable) > 0:
        hits += 1
        drawn_column = f"draw_{hits}"

        # hitting once
        print("simulating hit")
        playable = player_hits_once(playable.copy(), keep_drawn_column=True)
        playable = playable.rename(columns={"drawn":drawn_column})
        print("first_hit\n", playable)

        # expected return of hitting once
        standgame = dealer_plays(playable.copy())
        returncol = get_return_column(standgame)
        standgame["return"] = returncol
        standgame["expected_return"] = standgame["chance"] * standgame["return"]
        # print("\nstandgame\n", standgame[standgame["drawn_first"] == 11].sort_values("chance", ascending=False))

        standgame_grouped = standgame.groupby(drawn_column).agg({"expected_return":"sum","chance":"sum"}).reset_index().sort_values(drawn_column)
        print("standgame_grouped\n", standgame_grouped)
        print("expected_return_standing (calc old)\n", get_return_standing(playable))
        # this is the return of hitting exactly once. but maybe hitting again is better in some cases

        hitgame = player_hits_once(playable.copy(), keep_drawn_column=True)
        hitgame = hitgame.rename(columns={"drawn":"drawn_second"})
        print(hitgame)
        hitgame = dealer_plays(hitgame)
        returncol = get_return_column(hitgame)
        hitgame["return"] = returncol
        hitgame["expected_return"] = hitgame["chance"] * hitgame["return"]
        # print("\nhitgame\n", hitgame)
        hitgame_grouped = hitgame.groupby("drawn_first").agg({"expected_return":"sum","chance":"sum"}).reset_index().sort_values("drawn_first")
        print("hitgame grouped\n", hitgame_grouped)
        # print("expected_return_hitting_once\n", get_return_hitting_once(game))

        bestgame_grouped = standgame_grouped[["drawn_first","chance"]].copy()
        bestgame_grouped["stand"] = standgame_grouped["expected_return"]
        bestgame_grouped["hit"] = hitgame_grouped["expected_return"]
        bestgame_grouped["best"] = bestgame_grouped[["stand","hit"]].max(axis=1)
        bestgame_grouped["best_action"] = bestgame_grouped[["stand","hit"]].idxmax(axis=1)
        print("bestgame_grouped\n", bestgame_grouped)

    
    

In [107]:
def get_best_return(game, recursive=False):
    """get the expected return of the current state, assuming best play
    Only standing and hitting are implemented yet"""
    if not recursive:
        print(game)

    stand_return = get_return_standing(game)
    if not recursive:
        print("Stand return\n", stand_return)

    playable = game[(game["player_score"] >= 0) & (game["player_score"] != 22)].copy()

    if len(playable) == 0:
        return pd.Series({
        "stand":stand_return["expected_return"], 
        "hit":-1, 
        "best":stand_return["expected_return"],
        "option":"stand"
        })
    
    hit_game = player_hits_once(playable, keep_drawn_column=True)
    hit_game = hit_game.rename(columns={"drawn":"drawn_first"})
    hit_game_finished = dealer_plays(hit_game.copy())
    hit_game_finished["return"] = get_return_column(hit_game_finished)
    hit_game_finished["expected_return"] = hit_game_finished["chance"] * hit_game_finished["return"]
    

    hit_game_grouped = hit_game_finished.groupby("drawn_first").agg({"expected_return":"sum"}).reset_index().sort_values("drawn_first")
    if not recursive:
        print("Hit return\n", hit_game_grouped)
        print(hit_game_grouped["expected_return"].sum())

    for i in hit_game_grouped.index:
        drawn_card = hit_game_grouped.loc[i,"drawn_first"]
        

        if len(hit_game[(hit_game["player_score"] >= 0) & (hit_game["player_score"] != 22) & (hit_game["drawn_first"] == drawn_card)]) == 0 :
            continue # no need to simulate this draw, it is already busted
            
        if not recursive:
            print("sim", drawn_card)

        if hit_game[(hit_game["drawn_first"] == drawn_card)]["player_score"].min() == 21:
            if not recursive:
                print("is 21, so standing is better")
            continue # no need to simulate, hand is 21
        
        if not recursive:
            print("sim\n", hit_game[hit_game["drawn_first"] == drawn_card].copy().drop(columns=["drawn_first"]))
        hit_return = get_best_return(hit_game[hit_game["drawn_first"] == drawn_card].copy().drop(columns=["drawn_first"]), recursive=True)

        if not recursive:
            print("hitting after\n", hit_return)
        
        if hit_return["hit"] > hit_game_grouped.loc[i,"expected_return"] and not recursive:
            print("\nBetter return when hitting after", drawn_card, ":\n", hit_return, "\nvs\n", hit_game_grouped.loc[i])
        hit_game_grouped.loc[i,"expected_return"] = max(hit_return["hit"], hit_game_grouped.loc[i,"expected_return"])
    
    if not recursive:
        print("Updated hit return\n", hit_game_grouped)
        print("expected return", hit_game_grouped["expected_return"].sum())
        print("Stand return\n", stand_return)

    return_series = pd.Series({
        "stand":stand_return["expected_return"], 
        "hit":hit_game_grouped["expected_return"].sum(), 
        "best":max(stand_return["expected_return"], hit_game_grouped["expected_return"].sum()),
        "option":"stand" if stand_return["expected_return"] > hit_game_grouped["expected_return"].sum() else "hit"
        })
    return return_series


In [110]:
game = get_game(None, [6,6], [3], update_deck=False)
r = get_best_return(game)
r

   chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11
0     1.0            12             2            0             3             1            0  24  24  24  24  24  24  24  24  96  24
Stand return
 blackjack_win_chance    0.00000
win_chance              0.37395
push_chance             0.00000
lose_chance             0.62605
expected_return        -0.25210
dtype: float64
Hit return
    drawn_first  expected_return
0            2        -0.019378
1            3        -0.019355
2            4        -0.019178
3            5        -0.008779
4            6         0.011624
5            7         0.031329
6            8         0.050230
7            9         0.068093
8           10        -0.307692
9           11        -0.019283
-0.23238828771219874
sim 2
sim
      chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11
2  0.076923      

stand      -0.2521
hit      -0.232388
best     -0.232388
option         hit
dtype: object

In [112]:
print(pd.DataFrame({
    "standing":get_return_standing(game), 
    "hitting_once":get_return_standing(player_hits_once_where_possible(game)), 
    "hitting_twice":get_return_standing(player_hits_once_where_possible(player_hits_once_where_possible(game))), 
    "hitting_thrice":get_return_standing(player_hits_once_where_possible(player_hits_once_where_possible(player_hits_once_where_possible(game)))),
    "hitting_four_times":get_return_standing(player_hits_once_where_possible(player_hits_once_where_possible(player_hits_once_where_possible(player_hits_once_where_possible(game))))),
    "doubling":get_return_doubling_down(game)
    })
)

                      standing  hitting_once  hitting_twice  hitting_thrice  hitting_four_times  doubling
blackjack_win_chance  0.000000      0.000000       0.000000        0.000000            0.000000  0.000000
win_chance            0.426381      0.553199       0.295036        0.071599            0.012361  0.553199
push_chance           0.000000      0.062005       0.035860        0.009302            0.001619  0.062005
lose_chance           0.573619      0.384796       0.669104        0.919099            0.986020  0.384796
expected_return      -0.147238      0.168402      -0.374068       -0.847500           -0.973659  0.336805


# OLD

In [ ]:
game = get_game()

In [ ]:
game = dealer_draws_once(game)
game

,chance,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,5,6,7,8,9,10,11
0,0.076923,0,0,0,2,1,0,23,24,24,24,24,24,24,24,96,24
1,0.076923,0,0,0,3,1,0,24,23,24,24,24,24,24,24,96,24
2,0.076923,0,0,0,4,1,0,24,24,23,24,24,24,24,24,96,24
3,0.076923,0,0,0,5,1,0,24,24,24,23,24,24,24,24,96,24
4,0.076923,0,0,0,6,1,0,24,24,24,24,23,24,24,24,96,24
5,0.076923,0,0,0,7,1,0,24,24,24,24,24,23,24,24,96,24
6,0.076923,0,0,0,8,1,0,24,24,24,24,24,24,23,24,96,24
7,0.076923,0,0,0,9,1,0,24,24,24,24,24,24,24,23,96,24
8,0.307692,0,0,0,10,1,0,24,24,24,24,24,24,24,24,95,24
9,0.076923,0,0,0,11,1,1,24,24,24,24,24,24,24,24,96,23


In [ ]:
game = player_hits_once_where_possible(game)
game = player_hits_once_where_possible(game)
game

,chance,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,5,6,7,8,9,10,11
0,0.000404,4,2,0,2,1,0,21,24,24,24,24,24,24,24,96,24
1,0.000440,4,2,0,3,1,0,22,23,24,24,24,24,24,24,96,24
2,0.000440,4,2,0,4,1,0,22,24,23,24,24,24,24,24,96,24
3,0.000440,4,2,0,5,1,0,22,24,24,23,24,24,24,24,96,24
4,0.000440,4,2,0,6,1,0,22,24,24,24,23,24,24,24,96,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,0.003677,22,2,1,4,1,0,24,24,23,24,24,24,24,24,95,23
541,0.003677,22,2,1,3,1,0,24,23,24,24,24,24,24,24,95,23
548,0.014553,22,2,1,10,1,0,24,24,24,24,24,24,24,24,94,23
543,0.003677,22,2,1,5,1,0,24,24,24,23,24,24,24,24,95,23


In [ ]:
scores = get_return_standing(game, True)
scores

              chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11
0       1.427808e-04            17             2            0            17             2            1  24  24  24  24  23  24  23  23  96  23
1       5.711232e-04            12             2            0            17             2            0  24  23  24  24  24  23  24  23  95  24
2       1.368316e-04            12             2            0            17             2            0  24  23  24  24  24  24  23  22  96  24
3       1.427808e-04             9             2            0            17             2            0  24  23  24  24  23  24  23  23  96  24
4       5.711232e-04             9             2            0            17             2            0  24  24  23  23  24  23  24  24  95  24
...              ...           ...           ...          ...           ...           ...          ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..

blackjack_win_chance    0.047489
win_chance              0.339357
push_chance             0.046066
lose_chance             0.567088
expected_return        -0.156497
dtype: float64

In [ ]:
scores = get_return_hitting_once_where_possible(game, True)
scores

              chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11
0       6.397321e-05            19             3            0            17             2            0  24  24  24  22  24  23  24  23  95  24
1       1.532692e-05            19             3            0            17             2            0  24  24  24  22  24  24  23  22  96  24
2       1.335093e-04            19             3            0            17             2            0  24  24  24  23  23  23  23  24  95  24
3       3.198661e-05            19             3            0            17             2            0  24  24  24  23  23  24  22  23  96  24
4       5.864211e-05            19             3            0            17             2            0  24  24  24  23  24  21  24  24  95  24
...              ...           ...           ...          ...           ...           ...          ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..

blackjack_win_chance    0.047489
win_chance              0.264836
push_chance             0.049003
lose_chance             0.638671
expected_return        -0.302601
dtype: float64

In [ ]:
#TODO: implement splitting


To Beat:

r = get_best_return(get_game(None, [6,6], [3], update_deck=False))

1:30 min